### Import Library

In [1]:
import pandas as pd
import numpy as np
from linearmodels import PanelOLS,FamaMacBeth

In [2]:
import statsmodels.api as sm

### Import Data

In [3]:
data_directory = "data/"
file = "yahoo_data_filter_0_1.csv"

In [4]:
df = pd.read_csv(data_directory + file)
df["date"] = pd.to_datetime(df["date"])
df = df.set_index("date")

### Calculate the momentun signal

In [5]:
universe_df = df

In [6]:
momentum_period = 5
waiting_time = 1

In [7]:
complete_ret_sub_df = universe_df.pct_change(periods=momentum_period).shift(waiting_time).dropna()

In [8]:
complete_ret_sub_df_long = complete_ret_sub_df.stack()

### Create the future return

In [9]:
future_return_days = 1

In [10]:
future_return_df = universe_df.pct_change(future_return_days).shift(-(future_return_days-1))

In [11]:
future_return_df_long = future_return_df.stack()

### Merge Return with Signal

In [12]:
future_return_df_long = pd.DataFrame(future_return_df_long)
complete_ret_sub_df_long = pd.DataFrame(complete_ret_sub_df_long)

In [13]:
future_return_df_long.columns = ["target"]
complete_ret_sub_df_long.columns = ["signal"]

In [14]:
merge_df = future_return_df_long.merge(complete_ret_sub_df_long,left_index=True,right_index=True)

In [15]:
index_df = merge_df.index.to_frame()
index_df.columns = ["date","asset"]
index_df["target"] = merge_df["target"]
index_df["signal"] = merge_df["signal"]
index_df.index = range(0,index_df.shape[0])

In [16]:
merge_df = index_df

In [17]:
merge_df = merge_df.set_index(["asset","date"])

### Panel Regression

In [18]:
exog_vars = ["signal"]
depe_var = ["target"]
exog = sm.add_constant(merge_df[exog_vars])
depe = merge_df[depe_var]

In [19]:
mod = PanelOLS(depe,exog,entity_effects=True,time_effects=True)

In [20]:
mod.fit(cov_type="clustered",cluster_entity=True,cluster_time=True)

C:\app_user_download\anaconda\lib\site-packages\linearmodels\panel\results.py:84: RuntimeWarning: invalid value encountered in sqrt
  return Series(np.sqrt(np.diag(self.cov)), self._var_names, name="std_error")


Dep. Variable:,target,R-squared:,0.0007
Estimator:,PanelOLS,R-squared (Between):,-0.0523
No. Observations:,103645,R-squared (Within):,0.0004
Date:,"Thu, Jan 14 2021",R-squared (Overall):,0.0004
Time:,21:00:33,Log-likelihood,2.83e+04
Cov. Estimator:,Clustered,,
,,F-statistic:,73.082
Entities:,95,P-value,0.0000
Avg Obs:,1091.0,Distribution:,"F(1,102459)"
Min Obs:,1091.0,,
Max Obs:,1091.0,F-statistic (robust):,18.335


In [21]:
mod = PanelOLS(depe,exog,entity_effects=True,time_effects=True)

In [22]:
mod.fit(cov_type="kernel",kernel="parzen",cluster_entity=True,cluster_time=True)

Dep. Variable:,target,R-squared:,0.0007
Estimator:,PanelOLS,R-squared (Between):,-0.0523
No. Observations:,103645,R-squared (Within):,0.0004
Date:,"Thu, Jan 14 2021",R-squared (Overall):,0.0004
Time:,21:00:37,Log-likelihood,2.83e+04
Cov. Estimator:,Driscoll-Kraay,,
,,F-statistic:,73.082
Entities:,95,P-value,0.0000
Avg Obs:,1091.0,Distribution:,"F(1,102459)"
Min Obs:,1091.0,,
Max Obs:,1091.0,F-statistic (robust):,19.586


### Fama Macbeth Regression

In [27]:
exog_vars = ["signal"]
depe_var = ["target"]
exog = sm.add_constant(merge_df[exog_vars])
depe = merge_df[depe_var]

In [28]:
mod = FamaMacBeth(depe,exog)

In [34]:
res = mod.fit(cov_type='kernel',kernel="bartlett")

In [35]:
res

Dep. Variable:,target,R-squared:,-7.399e-05
Estimator:,FamaMacBeth,R-squared (Between):,-0.1069
No. Observations:,103645,R-squared (Within):,2.385e-05
Date:,"Sat, Jan 09 2021",R-squared (Overall):,-7.399e-05
Time:,16:26:11,Log-likelihood,2.486e+04
Cov. Estimator:,Fama-MacBeth Kernel Cov,,
,,F-statistic:,-7.6677
Entities:,95,P-value,1.0000
Avg Obs:,1091.0,Distribution:,"F(1,103643)"
Min Obs:,1091.0,,
Max Obs:,1091.0,F-statistic (robust):,10.434


### Multiple Signal Panel Regression

#### Create Multiple Signal 

In [38]:
momentum_period = 1
waiting_time = 1

In [39]:
complete_ret_sub_df_s1 = universe_df.pct_change(periods=momentum_period).shift(waiting_time).dropna()
complete_ret_sub_df_s1_long = complete_ret_sub_df_s1.stack()

In [40]:
momentum_period = 3
waiting_time = 1

In [41]:
complete_ret_sub_df_s3 = universe_df.pct_change(periods=momentum_period).shift(waiting_time).dropna()
complete_ret_sub_df_s3_long = complete_ret_sub_df_s3.stack()

In [42]:
momentum_period = 5
waiting_time = 1

In [43]:
complete_ret_sub_df_s5 = universe_df.pct_change(periods=momentum_period).shift(waiting_time).dropna()
complete_ret_sub_df_s5_long = complete_ret_sub_df_s5.stack()

In [44]:
momentum_period = 7
waiting_time = 1

In [45]:
complete_ret_sub_df_s7 = universe_df.pct_change(periods=momentum_period).shift(waiting_time).dropna()
complete_ret_sub_df_s7_long = complete_ret_sub_df_s7.stack()

#### Create Future Return

In [46]:
future_return_days = 1

future_return_df = universe_df.pct_change(future_return_days).shift(-(future_return_days-1))
future_return_df_long = future_return_df.stack()

future_return_df_long = pd.DataFrame(future_return_df_long)

#### Merge the dataframe

In [52]:
future_return_df_long = pd.DataFrame(future_return_df_long)

In [51]:
signal_df = pd.concat([complete_ret_sub_df_s1_long,
                       complete_ret_sub_df_s3_long,
                       complete_ret_sub_df_s5_long,
                       complete_ret_sub_df_s7_long],axis=1)

In [59]:
merge_df = future_return_df_long.merge(signal_df,left_index=True,right_index=True)

In [63]:
merge_df.columns = ["target","s1","s3","s5","s7"]



index_df = merge_df.index.to_frame()
index_df.columns = ["date","asset"]


index_df["target"] = merge_df["target"]
index_df["s1"] = merge_df["s1"]
index_df["s3"] = merge_df["s3"]
index_df["s5"] = merge_df["s5"]
index_df["s7"] = merge_df["s7"]

index_df.index = range(0,index_df.shape[0])

In [64]:
merge_df = index_df

merge_df = merge_df.set_index(["asset","date"])
merge_df = merge_df.dropna()

#### Panel Regression

In [69]:
exog_vars = ["s1","s3","s5","s7"]
depe_var = ["target"]
exog = sm.add_constant(merge_df[exog_vars])
depe = merge_df[depe_var]

In [70]:
mod = PanelOLS(depe,exog,entity_effects=True,time_effects=True)
mod.fit(cov_type="clustered",cluster_entity=True,cluster_time=True)

In [72]:
mod = PanelOLS(depe,exog,entity_effects=False,time_effects=False)

In [74]:
mod.fit(cov_type="clustered",cluster_entity=True,cluster_time=True)

Dep. Variable:,target,R-squared:,0.0023
Estimator:,PanelOLS,R-squared (Between):,-0.0755
No. Observations:,103455,R-squared (Within):,0.0023
Date:,"Thu, Jan 14 2021",R-squared (Overall):,0.0023
Time:,21:24:22,Log-likelihood,2.496e+04
Cov. Estimator:,Clustered,,
,,F-statistic:,58.860
Entities:,95,P-value,0.0000
Avg Obs:,1089.0,Distribution:,"F(4,103450)"
Min Obs:,1089.0,,
Max Obs:,1089.0,F-statistic (robust):,30.036


#### Multiple Fama Macbeth Regression

In [75]:
mod = FamaMacBeth(depe,exog)

In [76]:
res = mod.fit(cov_type='kernel',kernel="bartlett")

In [77]:
res

Dep. Variable:,target,R-squared:,-0.0123
Estimator:,FamaMacBeth,R-squared (Between):,-0.6027
No. Observations:,103455,R-squared (Within):,-0.0118
Date:,"Thu, Jan 14 2021",R-squared (Overall):,-0.0123
Time:,21:25:37,Log-likelihood,2.421e+04
Cov. Estimator:,Fama-MacBeth Kernel Cov,,
,,F-statistic:,-314.32
Entities:,95,P-value,1.0000
Avg Obs:,1089.0,Distribution:,"F(4,103450)"
Min Obs:,1089.0,,
Max Obs:,1089.0,F-statistic (robust):,26.607


### Multiple Holding Days

In [78]:
future_return_days = 2

future_return_df = universe_df.pct_change(future_return_days).shift(-(future_return_days-1))
future_return_df_long = future_return_df.stack()

future_return_df_long = pd.DataFrame(future_return_df_long)

In [79]:
merge_df = future_return_df_long.merge(signal_df,left_index=True,right_index=True)

In [80]:
merge_df

0        s1        s3        s5        s7
date                                                                 
2018-01-02 ADA-USD   0.481712  0.012453       NaN       NaN       NaN
           ADX-USD  -0.080739  0.114517       NaN       NaN       NaN
           AE-USD    0.175440 -0.015999       NaN       NaN       NaN
           ANT-USD   0.532966  0.098899       NaN       NaN       NaN
           ARDR-USD  0.215451 -0.065361       NaN       NaN       NaN
...                       ...       ...       ...       ...       ...
2021-01-02 XVG-USD   0.417309 -0.022395 -0.015002 -0.065067  0.100306
           XZC-USD  -0.045698 -0.094004 -0.056278 -0.142921 -0.056563
           ZEC-USD   0.029607 -0.112465 -0.146453 -0.160452 -0.086150
           ZEN-USD   0.182516  0.018508  0.018827 -0.059865  0.050730
           ZRX-USD   0.019493  0.058472  0.026020  0.004323  0.046617

[103930 rows x 5 columns]

In [81]:
merge_df

0        s1        s3        s5        s7
date                                                                 
2018-01-02 ADA-USD   0.481712  0.012453       NaN       NaN       NaN
           ADX-USD  -0.080739  0.114517       NaN       NaN       NaN
           AE-USD    0.175440 -0.015999       NaN       NaN       NaN
           ANT-USD   0.532966  0.098899       NaN       NaN       NaN
           ARDR-USD  0.215451 -0.065361       NaN       NaN       NaN
...                       ...       ...       ...       ...       ...
2021-01-02 XVG-USD   0.417309 -0.022395 -0.015002 -0.065067  0.100306
           XZC-USD  -0.045698 -0.094004 -0.056278 -0.142921 -0.056563
           ZEC-USD   0.029607 -0.112465 -0.146453 -0.160452 -0.086150
           ZEN-USD   0.182516  0.018508  0.018827 -0.059865  0.050730
           ZRX-USD   0.019493  0.058472  0.026020  0.004323  0.046617

[103930 rows x 5 columns]

In [82]:
merge_df.columns = ["target","s1","s3","s5","s7"]



index_df = merge_df.index.to_frame()
index_df.columns = ["date","asset"]


index_df["target"] = merge_df["target"]
index_df["s1"] = merge_df["s1"]
index_df["s3"] = merge_df["s3"]
index_df["s5"] = merge_df["s5"]
index_df["s7"] = merge_df["s7"]

index_df.index = range(0,index_df.shape[0])

In [83]:
merge_df = index_df

merge_df = merge_df.set_index(["asset","date"])
merge_df = merge_df.dropna()

#### Panel Regression

In [84]:
exog_vars = ["s1","s3","s5","s7"]
depe_var = ["target"]
exog = sm.add_constant(merge_df[exog_vars])
depe = merge_df[depe_var]

In [85]:
mod = PanelOLS(depe,exog,entity_effects=True,time_effects=True)
mod.fit(cov_type="clustered",cluster_entity=True,cluster_time=True)

C:\app_user_download\anaconda\lib\site-packages\linearmodels\panel\results.py:84: RuntimeWarning: invalid value encountered in sqrt
  return Series(np.sqrt(np.diag(self.cov)), self._var_names, name="std_error")


Dep. Variable:,target,R-squared:,0.0018
Estimator:,PanelOLS,R-squared (Between):,-0.0708
No. Observations:,103360,R-squared (Within):,0.0014
Date:,"Thu, Jan 14 2021",R-squared (Overall):,0.0013
Time:,21:32:21,Log-likelihood,4.906e+04
Cov. Estimator:,Clustered,,
,,F-statistic:,46.153
Entities:,95,P-value,0.0000
Avg Obs:,1088.0,Distribution:,"F(4,102174)"
Min Obs:,1088.0,,
Max Obs:,1088.0,F-statistic (robust):,23.508


In [86]:
mod = PanelOLS(depe,exog,entity_effects=False,time_effects=False)
mod.fit(cov_type="clustered",cluster_entity=False,cluster_time=True)

Dep. Variable:,target,R-squared:,0.0016
Estimator:,PanelOLS,R-squared (Between):,-0.0573
No. Observations:,103360,R-squared (Within):,0.0017
Date:,"Thu, Jan 14 2021",R-squared (Overall):,0.0016
Time:,21:33:02,Log-likelihood,4.094e+04
Cov. Estimator:,Clustered,,
,,F-statistic:,41.770
Entities:,95,P-value,0.0000
Avg Obs:,1088.0,Distribution:,"F(4,103355)"
Min Obs:,1088.0,,
Max Obs:,1088.0,F-statistic (robust):,2.5268


In [88]:
mod = PanelOLS(depe,exog,entity_effects=False,time_effects=False)
mod.fit(cov_type="clustered",cluster_entity=True,cluster_time=True)

Dep. Variable:,target,R-squared:,0.0016
Estimator:,PanelOLS,R-squared (Between):,-0.0573
No. Observations:,103360,R-squared (Within):,0.0017
Date:,"Thu, Jan 14 2021",R-squared (Overall):,0.0016
Time:,21:33:32,Log-likelihood,4.094e+04
Cov. Estimator:,Clustered,,
,,F-statistic:,41.770
Entities:,95,P-value,0.0000
Avg Obs:,1088.0,Distribution:,"F(4,103355)"
Min Obs:,1088.0,,
Max Obs:,1088.0,F-statistic (robust):,8.2934


#### Fama Macbeth

In [89]:
mod = FamaMacBeth(depe,exog)

In [90]:
mod.fit(cov_type='kernel',kernel="bartlett")

Dep. Variable:,target,R-squared:,-0.0110
Estimator:,FamaMacBeth,R-squared (Between):,-0.5999
No. Observations:,103360,R-squared (Within):,-0.0104
Date:,"Thu, Jan 14 2021",R-squared (Overall):,-0.0110
Time:,21:35:15,Log-likelihood,4.029e+04
Cov. Estimator:,Fama-MacBeth Kernel Cov,,
,,F-statistic:,-280.84
Entities:,95,P-value,1.0000
Avg Obs:,1088.0,Distribution:,"F(4,103355)"
Min Obs:,1088.0,,
Max Obs:,1088.0,F-statistic (robust):,15.834
